Setup

In [1]:
import benepar
import spacy
import pandas as pd
import decimal
import re
from lxml import etree

import apted
from apted import APTED
from apted.helpers import Tree
from itertools import combinations
from math import comb

import time
import json
from datetime import datetime
import os

benepar.download('benepar_en3')

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

TED_COMBINATIONS = 0
TED_ADJACENT = 1

AOA_AVG = 0
AOA_MAX = 0

C:\Users\cedch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\cedch\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


Helper Methods

In [9]:
# SEXP TO XML
def clean_xml(xml):
    xml = re.sub('<(/?)[^a-zA-Z/][^>]*>', '<\g<1>UNK>', xml) # invalid tokens labeled 'UNK'
    return xml.replace(' ', '')

def sexp_to_xml(sexp):
    def apply_inner_re(s):
        return re.sub('\(([^ ]*) ([^\)\(]*)\)', '<\g<1>> \g<2> </\g<1>>', s)

    xml = apply_inner_re(sexp)
    while xml.startswith('('):
        xml = apply_inner_re(xml)

    with open('special_chars.txt') as f:
        special_chars = dict([line.split() for line in f])

    def key_to_re(s):
        s = re.sub('(.*)([\\\.\+\*\?\^\$\(\)\[\]\{\}\|])(.*)', '\g<1>\\\\\g<2>\g<3>', s)
        return '<(/?)' + s + '>'

    for k, v in special_chars.items():
        xml = re.sub(key_to_re(k), f'<\g<1>{v}>', xml)

    return clean_xml(xml)

# TREE EDIT DISTANCE
def apted_format(parse_str):
    parse_str = re.sub('\(([^ ]+) [^ \(\)]+?\)', '(\g<1>)', parse_str)
    parse_str = parse_str.replace(' ', '')
    parse_str = parse_str.replace('(', '{')
    parse_str = parse_str.replace(')', '}')
    return parse_str

# PARATACTIC CHILDREN STRICT 
def find_parataxis_strict(e):
    global clause_tags
    
    children = [c.tag for c in e.getchildren() if not(c.tag.startswith('PUNCT-'))] # excludes punct
    sum = 0
    in_group = False
    
    for i in range(len(children) - 1):
        if children[i] in clause_tags and children[i + 1] in clause_tags:
            sum += 1
            if not(in_group):
                sum += 1
                in_group = True
        else:
            in_group = False
    return sum

# LEMMATIZATION
def lemmatize(word: str):
    return lemmatizer(word)[0].lemma_

# AOA
def aoa_of(word: str):
    global aoa_df, aoa_mode
    
    search = aoa_df[aoa_df['Word'] == word]['Rating.Mean']
    if len(search) == 1:
        return float(search)
    
    lemma_search = aoa_df[aoa_df['Word'] == lemmatize(word)]['Rating.Mean']
    if len(lemma_search) == 1:
        return float(lemma_search)
    
    lemmas = aoa_df[aoa_df['Lemma'] == lemmatize(word)]['Rating.Mean']
    if len(lemmas) == 0:
        return -1
    elif len(lemmas) == 1:
        return float(lemmas)
    elif aoa_mode == 'avg':
        return sum(lemmas) / len(lemmas)
    elif aoa_mode == 'max':
        return max(lemmas)
    
    return -1

#Average
def list_avg(l):
    return sum(l) / len(l) if len(l) > 0 else 0


Analysis Functions

In [10]:
clause_tags = ['S', 'SBARQ', 'SINV'] # Not included: 'SQ', 'SBAR'
clause_re = re.compile('(' + '/|'.join(clause_tags) + ')')
pronoun_tags = ['PRP', 'PRPS']

aoa_df = pd.read_csv('reference/aoa/aoa_lemmas.csv')

with open('reference/word_frequency/subtlexus_lower.json', encoding='utf-8') as f:
    wf_dict = json.load(f)

def benepar_analysis(sent):
    global num_clauses, num_sbar, num_unk, depth_sum, max_depth, max_clause_depth, clause_depth_sum
    global pronoun_sum, num_leaf_nps, num_nps, np_leaf_sum, clause_length_sum
    global paratactic_sum_loose, paratactic_sum_strict

    xml = sexp_to_xml(sent._.parse_string)
    root = etree.fromstring(xml) 
    tree = etree.ElementTree(root)

    num_sbar += sum(int(e.tag == 'SBAR') for e in root.iter())
    pronoun_sum += sum(int(e.tag in pronoun_tags) for e in root.iter())
    num_unk += sum(int(e.tag == 'UNK') for e in root.iter())
    for e in root.iter():
        tag = e.tag
        paratactic_sum_strict += find_parataxis_strict(e)
        if tag in clause_tags: 
            num_clauses += 1
            clause_length_sum += sum(int(not(d.tag.startswith('PUNCT-') and bool(d.text))) for d in e.iterdescendants())
            
            paratactic_sum_loose += sum(int(bool(c.tag in clause_tags)) for c in e.getchildren())
        elif e.tag == 'NP':
            num_nps += 1
            is_leaf_np = True
            for c in e.iterdescendants():
                if c.text: 
                    if not(c.tag.startswith('PUNCT-') or c.tag == 'DT'): # ignore determiners and punctuation
                        np_leaf_sum += 1
                else:
                    is_leaf_np = False
            if is_leaf_np:
                num_leaf_nps += 1
        
        if e.text:
            path = tree.getpath(e)

            depth = len(re.findall('/', path))
            depth_sum += depth # Number of times '/' appears, excluding first
            max_depth = max(max_depth, depth)

            clause_depth = len(clause_re.findall(path))
            clause_depth_sum += clause_depth
            max_clause_depth = max(max_clause_depth, clause_depth)

def spacy_analysis(sent):
    global dep_dist_sum, num_words, words_before_root_sum, uniq_words, num_words_no_nums, num_stop_words
    global max_dep_dist
    global aoa_list, aoa_stopless_list, aoa_uniq_list, aoa_stopless_uniq_list
    global wf_list, wf_stopless_list, wf_uniq_list, wf_stopless_uniq_list

    num_stop_words += sum(int(token.is_stop) for token in sent)

    for token in sent:
        if not(token.is_punct or token.is_space):
            num_words += 1
            dep_dist = abs(token.head.i - token.i)
            dep_dist_sum += dep_dist
            max_dep_dist = max(dep_dist, max_dep_dist)
            
            if (wf := wf_dict.get(token.lower_)) is not None:
                wf_list.append(wf)
                if not token.is_stop:
                    wf_stopless_list.append(wf)

            if (aoa := aoa_of(token.lower_)) != -1:
                aoa_list.append(aoa)
                if not token.is_stop:
                    aoa_stopless_list.append(aoa)

            if token.i < sent.root.i: words_before_root_sum += 1

            if not(token.like_num):
                if not(token.lower_ in uniq_words):
                    uniq_words.append(token.lower_)

                    if wf is not None:
                        wf_uniq_list.append(wf)
                        if not token.is_stop:
                            wf_stopless_uniq_list.append(wf)
                    
                    if aoa != -1:
                        aoa_uniq_list.append(aoa)
                        if not token.is_stop:
                            aoa_stopless_uniq_list.append(aoa)

                num_words_no_nums += 1
    
def ted_analysis(sent1, sent2):
    global ted_sum

    tree1 = Tree.from_text(apted_format(sent1._.parse_string))
    tree2 = Tree.from_text(apted_format(sent2._.parse_string))

    apted = APTED(tree1, tree2, )
    ted = apted.compute_edit_distance()
    ted_sum += ted

Main

In [11]:
results = pd.DataFrame([])

# TED Mode
#  - 'combinations': Averages the TED of every pair of sentences in a doc
#  - 'adjacent': Averages the TED of a sentence and the next sentence in a doc
ted_mode = 'adjacent'
# AoA Mode: If word is not in AoA list and multiple lemma matches exist
#  - 'max': Chooses max of lemma matches
#  - 'avg': Averages lemma matches
aoa_mode = 'max'
log = ""

for i, file in enumerate(os.scandir('data/text_jsons/')):
    if i == 2: break
    
    file_time = time.perf_counter()
    file_name = re.sub('\.json$', '', file.name)
    with open(file, encoding='utf-8') as f:
        metadata = json.load(f)
    text = metadata['text']
    text = text.replace('\n', ' ').strip()
    text = re.sub('\s{2,}', ' ', text)

    try:
        doc = nlp(text)
    except ValueError:
        error_text = f"ValueError in '{file_name}'. Likely exists too long sentence. Skipping."
        log += error_text + '\n'
        continue
    except:
        error_text = f"Some other error occured in '{file_name}'. Skipping."
        print(error_text)
        log += error_text + '\n'
        continue

    sents = list(doc.sents)

    # Doc-level
    num_tokens = len(doc)
    num_sents = len(sents)
    ted_sum = 0

    # Benepar
    num_clauses = 0
    num_sbar = 0
    num_unk = 0
    depth_sum = 0
    max_depth = 0
    clause_depth_sum = 0
    max_clause_depth = 0
    clause_length_sum = 0
    pronoun_sum = 0
    num_leaf_nps = 0 # all children are leaves
    num_nps = 0 # all NPs
    np_leaf_sum = 0 # number of leaf descendents a NP has (i.e., number of modifying words)
    paratactic_sum_strict = 0 # excludes coordinating conjunctions
    paratactic_sum_loose = 0 # includes

    # spaCy
    dep_dist_sum = 0
    max_dep_dist = 0
    num_words = 0
    num_words_no_nums = 0
    uniq_words = []
    words_before_root_sum = 0 # Root as in word whose head is self
    num_stop_words = 0
    aoa_list = []
    aoa_uniq_list = []
    aoa_stopless_list = []
    aoa_stopless_uniq_list = []
    wf_list = []
    wf_uniq_list = []
    wf_stopless_list = []
    wf_stopless_uniq_list = []

    for sent in sents:
        benepar_time = time.perf_counter()
        benepar_analysis(sent)
        benepar_time = time.perf_counter() - benepar_time

        spacy_time = time.perf_counter()
        spacy_analysis(sent)
        spacy_time = time.perf_counter() - spacy_time

    # TREE EDIT DISTANCE
    ted_time = time.perf_counter()
    if ted_mode == 'adjacent':    
        for i in range(num_sents - 1):
            ted_analysis(sents[i], sents[i + 1])
        ted_avg = ted_sum / (num_sents - 1)
    elif ted_mode == 'combinations':
        for sent1, sent2 in combinations(sents, 2):
            ted_analysis(sent1, sent2)
        ted_avg = ted_sum / comb(num_sents, 2)
    else:
        print('Invalid ted_mode:', ted_mode)
        ted_avg = -1
    ted_time = time.perf_counter() - ted_time

    summary = {
        # File-level
        'date' : metadata['date'],
        'pres_name' : metadata['pres_name'],
        'byline' : metadata['byline'],
        'title' : metadata['title'],

        # Doc-level
        'num_tokens' : num_tokens,
        'num_sentences' : num_sents, 
        # 'avg_ted_adj' : ted_avg_adj,
        # 'avg_ted_comb' : ted_avg_comb,
        f'avg_tree_edit_dist_{ted_mode}' : ted_avg,

        # Benepar
        'avg_node_depth' : depth_sum / num_tokens, 
        'max_node_depth' : max_depth, # Equivalent to tree height
        'avg_node_clause_depth' : clause_depth_sum / num_tokens,
        'max_node_clause_depth' : max_clause_depth,
        'avg_clause_length' : clause_length_sum / num_clauses,
        'clauses_per_sent' : num_clauses / num_sents, 
        'sbars_per_sent' : num_sbar / num_sents,
        'pronouns_per_sent' : pronoun_sum / num_sents,
        'pronouns_per_clause' : pronoun_sum / num_clauses,
        'pronoun_prop_of_leaf_nps' : pronoun_sum / num_leaf_nps,
        'avg_num_np_modifiers' : np_leaf_sum / num_nps,
        'parataxis_per_sent_strict' : paratactic_sum_strict / num_sents,
        'parataxis_per_sent_loose' : paratactic_sum_loose / num_sents,
        'num_unk' : num_unk,

        # spaCy
        'num_words' : num_words,
        'avg_dependency_distance' : dep_dist_sum / num_words,
        'max_dependency_distance' : max_dep_dist,
        'avg_sentence_length_by_tok' : num_tokens / num_sents, 
        'avg_sentence_length_by_word' : num_words / num_sents,
        'avg_words_before_root' : words_before_root_sum / num_sents,
        'num_uniq_words' : len(uniq_words), 
        'proportion_uniq' : len(uniq_words) / num_words_no_nums,
        'stop_words_per_clause' : num_stop_words / num_clauses,
        'stop_words_per_sentence' : num_stop_words / num_sents,
        f'avg_aoa_{aoa_mode}' : list_avg(aoa_list),
        f'avg_aoa_uniq_{aoa_mode}' : list_avg(aoa_uniq_list),
        f'avg_stopless_aoa_{aoa_mode}' : list_avg(aoa_stopless_list),
        f'avg_stopless_aoa_uniq_{aoa_mode}' : list_avg(aoa_stopless_uniq_list),
        'avg_word_freq' : list_avg(wf_list),
        'avg_word_freq_uniq' : list_avg(wf_uniq_list),
        'avg_word_freq_stopless' : list_avg(wf_stopless_list),
        'avg_word_freq_stopless_uniq' : list_avg(wf_stopless_uniq_list),

        # Performance time
        'benepar_analysis_time' : benepar_time,
        'spacy_analysis_time' : spacy_time,
        'tree_edit_distance_time' : ted_time,
        'total_file_analysis_time' : time.perf_counter() - file_time
    }      
    
    results[file_name] = summary

print("Error Log:")
print(log)
pd.set_option('display.precision', 2)
print(results)
results.to_csv(f"results/{datetime.now().strftime('%m-%d-%Y_%H-%M')}.csv")


C:\Users\cedch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\distributions\distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +
C:\Users\cedch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\distributions\distribution.py:45: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


Error Log:

                                           1789_Washington_Inaugural_Address  \
date                                                              1789-04-30   
pres_name                                                  George Washington   
byline                       1st President of the United States: 1789 ‐ 1797   
title                                                      Inaugural Address   
num_tokens                                                              1546   
num_sentences                                                             23   
avg_tree_edit_dist_adjacent                                           138.23   
avg_node_depth                                                         12.68   
max_node_depth                                                            50   
avg_node_clause_depth                                                   2.46   
max_node_clause_depth                                                      8   
avg_clause_length           

- vocab measures
    - measures of polysemy
- weight ted differently --> should adding/removing be weighted less because it simply indicates a different length sentence?
    - how to remove determiners?
- make scraper more robust
- unit testing for parataxis measures

- word not in AoA list 
    - opt 1: average lemmas
    - opt 2: take max


Questions
- Workaround for the spacy max length? <code>ValueError: Sentence of length 965 (in sub-word tokens) exceeds the maximum supported length of 512</code>
- How to compare syntax trees better?
    - Should I do TED weighting?
- Visually displaying / analyzing the data
- Code not running on harris...

- View: https://scholarworks.gsu.edu/cgi/viewcontent.cgi?article=1035&context=alesl_diss
- View: http://cohmetrix.memphis.edu/cohmetrixhome/documentation_indices.html#Complexity
- MORE RESEARCH NEEDED: compare tree similarity (SEARCH TREE EDIT DISTANCE) of sentences in doc, pq-gram distance
(CITING APTED: https://pypi.org/project/apted/#description)

In [ ]:
! pip install benepar
! pip install spacy
! pip install apted
! pip install bs4
! python -m spacy download en_core_web_md


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\cedch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\cedch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\cedch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\cedch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 42.8/42.8 MB 5.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: C:\Users\cedch\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
